In [11]:
import random
import string

class TimeTableGA:
    def __init__(self, population_size, mutation_rate, crossover_rate, 
                 days=None, time_slots=None, professors=None, courses=None, sections=None):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.days = days if days else self._generate_random_days()
        self.time_slots = time_slots if time_slots else self._generate_random_time_slots()
        print(self.time_slots)
        self.rooms = {
            'classroom': 60,
            'labs': 120
        }
        self.professors = professors if professors else self._generate_random_professors()
        self.courses = courses if courses else self._generate_random_courses()
        self.sections = sections if sections else self._generate_random_sections()
        # Initialize population
        self.population = self._initialize_population()

    def _generate_random_days(self):
        #Generate random days.
        return random.sample(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], 5)

    def _generate_random_time_slots(self):
        #Generate random time slots.
        return [f"{2 * hour}:00-{2 * hour + 1}:30" for hour in range(6)]


    def _generate_random_professors(self):
        #Generate random professors.
        return ['Prof_' + ''.join(random.choices(string.ascii_uppercase, k=5)) for _ in range(20)]

    def _generate_random_courses(self):
        #Generate random courses.
        return ['Course_' + ''.join(random.choices(string.ascii_uppercase, k=3)) for _ in range(16)]

    def _generate_random_sections(self):
        #Generate random sections.
        return ['Section_' + ''.join(random.choices(string.ascii_uppercase, k=1)) for _ in range(10)]

    def _initialize_population(self):
        #Generate initial random population of timetables.
        population = []
        for _ in range(self.population_size):
            timetable = self._generate_random_timetable()
            population.append(timetable)
        return population

    def _generate_random_timetable(self):
        #Generate a random timetable.
        timetable = {}

        for day in self.days:
            for time_slot in self.time_slots:
                timetable[(day, time_slot)] = {
                    'room': random.choice(list(self.rooms.keys())),
                    'course': random.choice(self.courses),
                    'professor': random.choice(self.professors),
                    'section': random.choice(self.sections)
                }
        return timetable

    def _get_next_time_slot(self, time_slot):
        index = self.time_slots.index(time_slot)
        if index < len(self.time_slots) - 1:
            return self.time_slots[index + 1]
        else:
            return None


    def _fitness_function(self, timetable):
        #Fitness function to evaluate the timetable based on various constraints.
        #Returns the negative of the total number of conflicts.
        #Constraint 1: Empty Classroom is checked by Constraint 4 and 5
        conflicts = 0
        professors_schedule = {}
        professor_course_count = {professor: 0 for professor in self.professors}

        section_course_count = {section: 0 for section in self.sections}

        lab_slots = set()

        classrooms_schedule = {classroom: set() for classroom in self.rooms.keys()}



        for slot, info in timetable.items():
            professor = info['professor']
            section = info['section']
            room = info['room']

            #Constraint 3: No 2 profoessors can be in same slot
            if professor in professors_schedule:
                if slot in professors_schedule[professor]:
                    conflicts += 1
                else:
                    professors_schedule[professor].append(slot)
            else:
                professors_schedule[professor] = [slot]


            #Constraint 7: No more than 5 courses per section
            section_course_count[section] += 1
            if section_course_count[section] > 5:
                conflicts += 1
                
            # Constraint 9: Lab lectures should be conducted in two consecutive slots
            course = info['course']
            if course.startswith('Lab'):
                if slot in lab_slots:
                    lab_slots.remove(slot)
                else:
                    lab_slots.add(slot)
            else:
                if (slot[0], self._get_next_time_slot(slot[1])) in lab_slots:
                    lab_slots.remove((slot[0], self._get_next_time_slot(slot[1])))
                else:
                    conflicts += 1

            #Constraint 6: No more than 3 courses per professor
            professor_course_count[professor] += 1
            if professor_course_count[professor] > 3:
                conflicts += 1

            if slot in classrooms_schedule[room]:
                conflicts += 1
            else:
                classrooms_schedule[room].add(slot)

        # Constraint 4: The same section cannot be assigned to two different rooms at the same time
        sections_schedule = {}
        for slot, info in timetable.items():
            section = info['section']
            if section in sections_schedule:
                if slot in sections_schedule[section]:
                    conflicts += 1
                else:
                    sections_schedule[section].append(slot)
            else:
                sections_schedule[section] = [slot]

        # Constraint 5: A room cannot be assigned for two different sections at the same time
        rooms_schedule = {}
        for slot, info in timetable.items():
            room = info['room']
            if room in rooms_schedule:
                for scheduled_slot in rooms_schedule[room]:
                    if slot[0] == scheduled_slot[0] and self._is_time_overlap(slot[1], scheduled_slot[1]):
                        conflicts += 1
                        break
                else:
                    rooms_schedule[room].append(slot)
            else:
                rooms_schedule[room] = [slot]

        # Soft Constraint : All the theory classes should be taught in the morning session
        for day in self.days:
            for time_slot in self.time_slots[:3]:
                for slot, info in timetable.items():
                    if slot[0] == day and slot[1] == time_slot:
                        if info['course'].startswith('Lab'):
                            conflicts += 1

        # Soft Constraint : All the lab sessions should be done in the afternoon session
        for day in self.days:
            for time_slot in self.time_slots[3:]:
                for slot, info in timetable.items():
                    if slot[0] == day and slot[1] == time_slot:
                        if not info['course'].startswith('Lab'):
                            conflicts += 1

        # Constraint 8: Each course would have two lectures per week not on the same or adjacent days
        course_schedule = {course: [] for course in self.courses}
        for slot, info in timetable.items():
            course = info['course']
            course_schedule[course].append(self.days.index(slot[0]))
        for course, days in course_schedule.items():
            days.sort()
            for i in range(len(days) - 1):
                if days[i + 1] == days[i] or days[i + 1] - days[i] == 1:
                    conflicts += 1

        # Constraint 10: Ensure 15 mins breaks between consecutive classes
        sorted_slots = sorted(timetable.keys(), key=lambda x: (self.days.index(x[0]), self.time_slots.index(x[1])))

        for i in range(1, len(sorted_slots)):
            current_slot = sorted_slots[i]
            previous_slot = sorted_slots[i - 1]
            current_day_index = self.days.index(current_slot[0])
            previous_day_index = self.days.index(previous_slot[0])
            current_time = list(map(int, current_slot[1].split('-')[1].split(':')))
            previous_time = list(map(int, previous_slot[1].split('-')[0].split(':')))
            current_minutes = current_time[0] * 60 + current_time[1]
            previous_minutes = previous_time[0] * 60 + previous_time[1]

            if current_day_index == previous_day_index and current_minutes - previous_minutes < 15:
                conflicts += 1

        # Return negative of conflicts as fitness value
        return -conflicts

    def _is_time_overlap(self, time1, time2):
        #Check if two time slots overlap.
        t1_start, t1_end = list(map(int, time1.split('-')[0].split(':'))), list(map(int, time1.split('-')[1].split(':')))
        t2_start, t2_end = list(map(int, time2.split('-')[0].split(':'))), list(map(int, time2.split('-')[1].split(':')))
        t1_start_minutes, t1_end_minutes = t1_start[0] * 60 + t1_start[1], t1_end[0] * 60 + t1_end[1]
        t2_start_minutes, t2_end_minutes = t2_start[0] * 60 + t2_start[1], t2_end[0] * 60 + t2_end[1]
        return not (t1_end_minutes <= t2_start_minutes or t1_start_minutes >= t2_end_minutes)

    def _select_parents(self):
        fitness_scores = [self._fitness_function(t) for t in self.population]
        min_fitness = min(fitness_scores)
        
        adjusted_fitness = [score - min_fitness + 1 for score in fitness_scores]
        return random.choices(self.population, weights=adjusted_fitness, k=2)

    def _crossover(self, parent1, parent2):
        all_children = []
        for slot in parent1.keys():
            child = {}
            for key in parent1.keys():
                if random.random() < self.crossover_rate:
                    child[key] = parent1[key]
                else:
                    child[key] = parent2[key]
            all_children.append(child)
    
        children_fitness = [(child, self._fitness_function(child)) for child in all_children]
    
        best_child, best_fitness = max(children_fitness, key=lambda x: x[1])
        return best_child

    def _mutate(self, timetable):
        mutated_timetable = timetable.copy()
        slot_to_mutate = random.choice(list(mutated_timetable.keys()))
        mutated_timetable[slot_to_mutate]['course'] = random.choice(self.courses)
        mutated_timetable[slot_to_mutate]['professor'] = random.choice(self.professors)
        mutated_timetable[slot_to_mutate]['room'] = random.choice(list(self.rooms.keys()))
        mutated_timetable[slot_to_mutate]['section'] = random.choice(self.sections)
        return mutated_timetable

    def evolve(self):
        # Sort the population based on fitness scores
        sorted_population = sorted(self.population, key=self._fitness_function, reverse=True)
        # Select top 50% of the fittest individuals
        top_half_population = sorted_population[:len(sorted_population) // 2]
        new_population = []
        for _ in range(self.population_size):
            # Select parents from the top 50% of the population
            parent1, parent2 = random.choices(top_half_population, k=2)
            # Perform crossover
            child = self._crossover(parent1, parent2)
            # Perform mutation
            if random.random() < self.mutation_rate:
                child = self._mutate(child)
            new_population.append(child)
        # Append new children to the existing population
        self.population.extend(new_population)


    def driver(self, generations):
        for gen in range(1, generations + 1):
            self.evolve()
            best_timetable = max(self.population, key=self._fitness_function)
            max_fitness = self._fitness_function(best_timetable)
            print(f"Generation {gen}: Maximum Fitness = {max_fitness}")
        best_timetable = max(self.population, key=self._fitness_function)
        print("\nBest Timetable:", best_timetable)

if __name__ == "__main__":
    scheduler = TimeTableGA(population_size=150, mutation_rate=0.1, crossover_rate=0.8)
    scheduler.driver(generations=100)


['0:00-1:30', '2:00-3:30', '4:00-5:30', '6:00-7:30', '8:00-9:30', '10:00-11:30']
Generation 1: Maximum Fitness = -50
Generation 2: Maximum Fitness = -50
Generation 3: Maximum Fitness = -49
Generation 4: Maximum Fitness = -49
Generation 5: Maximum Fitness = -48
Generation 6: Maximum Fitness = -48
Generation 7: Maximum Fitness = -48
Generation 8: Maximum Fitness = -48
Generation 9: Maximum Fitness = -47
Generation 10: Maximum Fitness = -47
Generation 11: Maximum Fitness = -47
Generation 12: Maximum Fitness = -47
Generation 13: Maximum Fitness = -47
Generation 14: Maximum Fitness = -47
Generation 15: Maximum Fitness = -46
Generation 16: Maximum Fitness = -46
Generation 17: Maximum Fitness = -47
Generation 18: Maximum Fitness = -48
Generation 19: Maximum Fitness = -48
Generation 20: Maximum Fitness = -48
Generation 21: Maximum Fitness = -48
Generation 22: Maximum Fitness = -48
Generation 23: Maximum Fitness = -48
Generation 24: Maximum Fitness = -47
Generation 25: Maximum Fitness = -47
Gen